In [1]:
# Some feature-engineering ideas were inspired by the following kernel: https://www.kaggle.com/siavrez/2020fatures

import numpy as np 
import pandas as pd
import re
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pd.set_option('max_rows', 300)
pd.set_option('display.max_columns', 300)
np.random.seed(666)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20,.3f}'.format)
pd.set_option('display.max_colwidth', None)

### Define functions and globals

In [2]:
def reduce_mem_usage(df: pd.DataFrame,
                     verbose: bool = True) -> pd.DataFrame:
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':

                if (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f'Mem. usage decreased to {end_mem:5.2f} MB ({reduction * 100:.1f} % reduction)'
    if verbose:
        print(msg)

    return df

In [3]:
TARGET_COL = "diabetes_mellitus"
RATIO_CAP = 2

# Data Loading

### Read in data and data dictionary

In [4]:
train_raw = pd.read_csv("Raw_Data/TrainingWiDS2021.csv")
test_raw = pd.read_csv("Raw_Data/UnlabeledWiDS2021.csv")
dd = pd.read_csv("Raw_Data/DataDictionaryWiDS2021.csv")
age_bmi_chart = pd.read_csv('age_bmi_chart.csv')

In [5]:
train = train_raw.drop('Unnamed: 0',axis=1)
test = test_raw.drop('Unnamed: 0',axis=1)
dd = dd[dd['Variable Name'] != 'icu_admit_type']

### Combine train and test sets to apply operations consistently across both sets

In [6]:
train['dataset_label'] = 0
test['dataset_label'] = 1
test[TARGET_COL] = np.nan

In [7]:
combined = pd.concat([train,test],ignore_index=True)
combined['hospital_admit_source'] = combined['hospital_admit_source'].replace({
    'Other ICU':'ICU','ICU to SDU':'SDU', 'Step-Down Unit (SDU)': 'SDU',
    'Other Hospital':'Other','Observation': 'Recovery Room'})
binary_vars = dd['Variable Name'][dd['Data Type'] == 'binary']
bool_vars = combined[binary_vars].isna().sum().index[combined[binary_vars].isna().sum() == 0]
combined[bool_vars] = combined[bool_vars].astype(bool)

# Data Cleaning

In [8]:
# checking if NA values consistently appear between min and max values (they should both be NA or both not be NA)
for col in [col for col in combined.columns if col.endswith('_min')]:
    col_max = re.sub('_min$','_max',col)
    if any(combined[[col,col_max]].isna().sum(axis=1) == 1):
        print(col)

# checking if NA values consistently appear between h1 and d1 values (if h1 value is NA then d1 value should also be NA)
for col in [col for col in combined.columns if col.startswith('d1_')]:
    col_h1 = re.sub('^d1_','h1_',col)
    if any(combined[col].isna() & combined[col_h1].notna()):
        print(col)

In [9]:
lab_tests = [re.sub('_min$','',re.sub('^d1_','',col)) for col in combined.columns if col.startswith('d1_') and col.endswith('_min')]

In [10]:
# orders = pd.DataFrame()
# for test in lab_tests:
#     test_d1_min = 'd1_'+test+'_min'
#     test_d1_max = 'd1_'+test+'_max'
#     test_h1_min = 'h1_'+test+'_min'
#     test_h1_max = 'h1_'+test+'_max'
#     ranks = pd.DataFrame(combined[[
#         test_d1_min,test_h1_min,test_h1_max,test_d1_max]].dropna().rank(axis=1,method='first').values)
#     orders = pd.concat([orders,ranks],ignore_index=True)
# orders = orders.astype(int).astype(str)
# (orders[0]+orders[1]+orders[2]+orders[3]).value_counts()

### Switching min and max values if min > max

In [11]:
for t in lab_tests:
    d1_min_test = 'd1_'+t+'_min'
    d1_max_test = 'd1_'+t+'_max'
    h1_min_test = 'h1_'+t+'_min'
    h1_max_test = 'h1_'+t+'_max'
    new_d1_min_test = combined[[d1_min_test,d1_max_test]].min(axis=1)
    new_d1_max_test = combined[[d1_min_test,d1_max_test]].max(axis=1)
    new_h1_min_test = combined[[h1_min_test,h1_max_test]].min(axis=1)
    new_h1_max_test = combined[[h1_min_test,h1_max_test]].max(axis=1)
    combined[t+'_d1_max_min_switched'] = (
        combined[d1_min_test] > new_d1_min_test) | (combined[d1_max_test] < new_d1_max_test)
    combined[t+'_h1_max_min_switched'] = (
        combined[h1_min_test] > new_h1_min_test) | (combined[h1_max_test] < new_h1_max_test)
    combined[t+'_any_max_min_switched'] = (combined[t+'_d1_max_min_switched'] | combined[t+'_h1_max_min_switched'])
    combined[d1_min_test] = new_d1_min_test
    combined[d1_max_test] = new_d1_max_test
    combined[h1_min_test] = new_h1_min_test
    combined[h1_max_test] = new_h1_max_test

In [ ]:
# corr_matrix = np.abs(combined.rank().corr())
# high_corrs_dict = {}
# corr_list = []
# for index, row in corr_matrix.iterrows():
#     corr_matrix.loc[index,index] = 0
#     corr_vars = list(row.index[row > .99])
#     if corr_vars:
#         high_corrs_dict[index] = corr_vars
#         corr_list.append(index)

# corr_matrix[corr_matrix != 1].max().sort_values(ascending=False)
# high_corrs_dict

### Add missing value count features for each category of variables as identified by the data dictionary

In [ ]:
# compute missing count variables
categories = dd.Category.unique()
categories = np.delete(categories, np.where(categories == 'identifier'))
categories = np.delete(categories, np.where(categories == 'Target Variable'))
for cat in categories:
    combined['num_na_'+cat] = combined[dd['Variable Name'][dd.Category == cat]].isna().sum(axis=1)
combined['num_na'] = combined.drop(TARGET_COL,axis=1).isna().sum(axis=1)

In [ ]:
# drop high corr and nonunique columns
extra_bp_vars = ['d1_diasbp_max','d1_diasbp_min','d1_sysbp_max','h1_sysbp_max','d1_sysbp_min','d1_mbp_min','h1_diasbp_max',
                 'h1_mbp_max','h1_mbp_min','h1_sysbp_min','d1_mbp_max','h1_diasbp_min']
combined = combined.drop(extra_bp_vars+['paco2_for_ph_apache','readmission_status'],axis=1)
lab_tests = list(set(lab_tests) - {'diasbp','sysbp','mbp'})

### Impute age, weight, and height using MICE, with separate imputations for males and females

In [ ]:
combined.loc[:,'age'].replace(0,np.nan,inplace=True)
imputerM = IterativeImputer(initial_strategy='median').fit(combined[['age','weight','height']][combined.gender == 'M'])
imputerF = IterativeImputer(initial_strategy='median').fit(combined[['age','weight','height']][combined.gender == 'F'])
combined[['age_Mimputed','weight_Mimputed','height_Mimputed']] = imputerM.transform(combined[['age','weight','height']])
combined[['age_Fimputed','weight_Fimputed','height_Fimputed']] = imputerF.transform(combined[['age','weight','height']])
combined['age'] = np.where(combined['age'].notna(), combined['age'],
                        np.where(combined['gender'] == 'M', combined['age_Mimputed'],
                        np.where(combined['gender'] == 'F', combined['age_Fimputed'],
                        np.nan)))
combined['weight'] = np.where(combined['weight'].notna(), combined['weight'],
                        np.where(combined['gender'] == 'M', combined['weight_Mimputed'],
                        np.where(combined['gender'] == 'F', combined['weight_Fimputed'],
                        np.nan)))
combined['height'] = np.where(combined['height'].notna(), combined['height'],
                        np.where(combined['gender'] == 'M', combined['height_Mimputed'],
                        np.where(combined['gender'] == 'F', combined['height_Fimputed'],
                        np.nan)))
rows_to_drop = (combined[['age','weight','height']].isna().sum(axis=1) > 0)
if len(combined[rows_to_drop & combined.dataset_label == 1]) == 0:
    combined = combined[~rows_to_drop]
combined.drop(['age_Mimputed','weight_Mimputed','height_Mimputed','age_Fimputed','weight_Fimputed','height_Fimputed'],
              axis=1,inplace=True)

### Create features based on blood pressure tests

In [ ]:
sysbp_tests = combined.columns[combined.columns.str.contains('sysbp') & ~combined.columns.str.endswith('_switched')]
for t in sysbp_tests:
    diasbp_test = re.sub('sysbp','diasbp',t)
    combined[re.sub('sysbp','sysbp_diasbp',t)+'_diff'] = combined[t] - combined[diasbp_test]
    combined[re.sub('sysbp','sysbp_diasbp',t)+'_ratio'] = np.maximum(
        np.minimum(combined[t] / combined[diasbp_test], RATIO_CAP), 0)
    combined[re.sub('sysbp','sysbp_diasbp',t)+'_diff_0_or_less'] = combined[re.sub('sysbp','sysbp_diasbp',t)+'_diff'] <= 0
for t in ['d1_sysbp_diasbp_invasive_max_diff','d1_sysbp_diasbp_noninvasive_max_diff',
          'h1_sysbp_diasbp_invasive_max_diff','h1_sysbp_diasbp_noninvasive_max_diff']:
    t_min = re.sub('_max_','_min_',t)
    combined[re.sub('_diff','_min_range_diff',t)] = combined[t] - combined[t_min]
    combined[re.sub('_diff','_min_range_ratio',t)] = np.maximum(np.minimum(combined[t] / combined[t_min], RATIO_CAP), 0)

### Create features based on logical operations on lab test features

In [ ]:
combined = combined.rename(columns={'pao2_apache':'pao2fio2ratio_apache','ph_apache':'arterial_ph_apache'})
for t in lab_tests:
    d1_min_test = 'd1_'+t+'_min'
    d1_max_test = 'd1_'+t+'_max'
    h1_min_test = 'h1_'+t+'_min'
    h1_max_test = 'h1_'+t+'_max'
    combined[t+'_d1_max_min_diff'] = combined[d1_max_test] - combined[d1_min_test]
    combined[t+'_h1_max_min_diff'] = combined[h1_max_test] - combined[h1_min_test]
    combined[t+'_d1_max_min_ratio'] = np.maximum(np.minimum(combined[d1_max_test] / combined[d1_min_test], RATIO_CAP), 0)
    combined[t+'_h1_max_min_ratio'] = np.maximum(np.minimum(combined[h1_max_test] / combined[h1_min_test], RATIO_CAP), 0)
    combined[t+'_d1_avg'] = (combined[d1_max_test] + combined[d1_min_test])/2
    combined[t+'_h1_avg'] = (combined[h1_max_test] + combined[h1_min_test])/2
    combined[t+'_max_d1_h1_diff'] = combined[d1_max_test] - combined[h1_max_test]
    combined[t+'_min_d1_h1_diff'] = combined[h1_min_test] - combined[d1_min_test]
    combined[t+'_max_d1_h1_ratio'] = np.maximum(np.minimum(combined[d1_max_test] / combined[h1_max_test], RATIO_CAP), 0)
    combined[t+'_min_d1_h1_ratio'] = np.maximum(np.minimum(combined[h1_min_test] / combined[d1_min_test], RATIO_CAP), 0)
    combined[t+'_max_h1_more_extreme'] = combined[t+'_max_d1_h1_diff'] < 0
    combined[t+'_min_h1_more_extreme'] = combined[t+'_min_d1_h1_diff'] < 0
    combined[t+'_any_h1_more_extreme'] = (combined[t+'_max_h1_more_extreme'] | combined[t+'_min_h1_more_extreme'])
    combined[t+'_d1_max_min_equal'] = (combined[t+'_d1_max_min_diff'] == 0)
    combined[t+'_h1_max_min_equal'] = (combined[t+'_h1_max_min_diff'] == 0)
    combined[t+'_max_d1_h1_equal'] = (combined[t+'_max_d1_h1_diff'] == 0)
    combined[t+'_min_d1_h1_equal'] = (combined[t+'_min_d1_h1_diff'] == 0)
    combined[t+'_max_or_min_d1_h1_equal'] = (combined[t+'_max_d1_h1_equal'] | combined[t+'_min_d1_h1_equal'])    
    combined[t+'_d1_h1_range_diff'] = combined[t+'_d1_max_min_diff'] - combined[t+'_h1_max_min_diff']
    combined[t+'_d1_h1_range_ratio'] = np.maximum(np.minimum(
        combined[t+'_d1_max_min_diff'] / combined[t+'_h1_max_min_diff'], RATIO_CAP), 0)
    combined[t+'_taken_d1'] = combined[[d1_min_test,d1_max_test]].notna().any(axis=1)
    combined[t+'_taken_h1'] = combined[[h1_min_test,h1_max_test]].notna().any(axis=1)
    combined[t+'_taken_only_after_h1'] = combined[t+'_taken_d1'] & ~combined[t+'_taken_h1']
    apache_test = t+'_apache'
    if apache_test in combined.columns:
        combined[t+'_d1_max_apache_diff'] = combined[d1_max_test] - combined[apache_test]
        combined[t+'_d1_min_apache_diff'] = combined[apache_test] - combined[d1_min_test]
        combined[t+'_d1_max_apache_ratio'] = np.maximum(np.minimum(combined[d1_max_test] / combined[apache_test], 2), 0)
        combined[t+'_d1_min_apache_ratio'] = np.maximum(np.minimum(combined[apache_test] / combined[d1_min_test], 2), 0)
        combined[t+'_d1_max_apache_equal_or_greater'] = (combined[t+'_d1_max_apache_diff'] <= 0)
        combined[t+'_d1_min_apache_equal_or_lesser'] = (combined[t+'_d1_min_apache_diff'] <= 0)
combined['any_max_min_switched'] = combined[combined.columns[combined.columns.str.endswith('_max_min_switched')]].any(axis=1)
combined['any_h1_more_extreme'] = combined[combined.columns[combined.columns.str.endswith('_h1_more_extreme')]].any(axis=1)
combined['num_d1_tests'] = combined[combined.columns[combined.columns.str.endswith('_taken_d1')]].sum(axis=1)
combined['num_h1_tests'] = combined[combined.columns[combined.columns.str.endswith('_taken_h1')]].sum(axis=1)
combined['num_d1_not_h1_tests'] = combined[combined.columns[combined.columns.str.endswith('_taken_only_after_h1')]].sum(axis=1)

### Create age and weight categories, and combine with ethnicity to create a demographic grouping variable

In [ ]:
combined['bmi'] = combined['weight']/((combined['height']/100)**2)
combined['age_range'] = pd.cut(combined.age, bins=[combined.age.min()-1,25,30,35,40,45,50,55,60,65,70,75,combined.age.max()+1], 
       right=False, labels=False)
combined = combined.merge(age_bmi_chart.drop('Age_Range',axis=1),left_on='age_range',right_on='Age_Index',how='left')
combined['weight_category'] = np.where(combined['bmi'] < combined['Percentile5'], 'Underweight',
                                    np.where(combined['bmi'] <= combined['Percentile85'], 'Healthy',
                                    np.where(combined['bmi'] <= combined['Percentile95'], 'Overweight',
                                    np.where(combined['bmi'] > combined['Percentile95'], 'Obese',''))))
combined['age_group'] = np.where(combined['age'] < 30, '18to29',
                            np.where(combined['age'] < 45, '30to44', 
                            np.where(combined['age'] < 65, '45to64',
                            np.where(combined['age'] >= 65, '65+', ''))))
combined['ethnic_group'] = np.where(combined.ethnicity == 'Caucasian', 'caucasian',
                                np.where(combined.ethnicity=='African American','african_american','other'))
combined['demo_profile'] = (combined['age_group'] + '_' + combined['weight_category'] + '_' + 
                            combined['ethnic_group'] + '_' + combined['gender'])
combined.drop(['ethnic_group','age_range','Age_Index','Percentile5','Percentile25','Percentile50',
               'Percentile75','Percentile85','Percentile95'],axis=1,inplace=True)

### Create grouping variables for glucose and blood pressure, as these are risk factors for diabetes, plus some misc. features

In [ ]:
combined['apache_3j_group'] = np.where(combined['apache_3j_diagnosis'].isna(), '',
                                    np.where(combined['apache_3j_diagnosis'] < 200, 'Cardiovascular' , 
                                    np.where(combined['apache_3j_diagnosis'] < 400, 'Respiratory' , 
                                    np.where(combined['apache_3j_diagnosis'] < 500, 'Neurological' , 
                                    np.where(combined['apache_3j_diagnosis'] < 600, 'Sepsis' , 
                                    np.where(combined['apache_3j_diagnosis'] < 800, 'Trauma' ,  
                                    np.where(combined['apache_3j_diagnosis'] < 900, 'Haematological' ,         
                                    np.where(combined['apache_3j_diagnosis'] < 1000, 'Renal/Genitourinary' ,         
                                    np.where(combined['apache_3j_diagnosis'] < 1200, 'Musculoskeletal/Skin disease',
                                             'Operative Sub-Diagnosis Codes' )))))))))
combined['gcs_sum'] = (combined['gcs_eyes_apache']+combined['gcs_motor_apache']+combined['gcs_verbal_apache']).fillna(0)
combined['comorbidity_score'] = (combined['aids'] * 23 + combined['cirrhosis'] * 4  + combined['hepatic_failure'] * 16 + 
                                 combined['immunosuppression'] * 10 + combined['leukemia'] * 10 + combined['lymphoma'] * 13 + 
                                 combined['solid_tumor_with_metastasis'] * 11).fillna(0)
combined['a1c'] = ((combined['glucose_d1_avg'])/2 + 46.7)/28.7
combined['a1c_group'] = np.where(combined['a1c'] < 5.7, 'Normal',
                                np.where(combined['a1c'] < 6.5, 'Prediabetes',
                                np.where(combined['a1c'] >= 6.5, 'Diabetes', 'Not_Measured')))
combined.drop('a1c',axis=1,inplace=True)
combined['bp_group'] = np.where(combined.sysbp_invasive_d1_avg.isna() &
                                combined.sysbp_noninvasive_d1_avg.isna() &
                                combined.diasbp_noninvasive_d1_avg.isna() &
                                combined.diasbp_invasive_d1_avg.isna(), 'Not_Measured',
                            np.where((combined.sysbp_invasive_d1_avg >= 140) | 
                                (combined.sysbp_noninvasive_d1_avg >= 140) |
                                (combined.diasbp_invasive_d1_avg >= 90) | 
                                (combined.diasbp_noninvasive_d1_avg >= 90), 'Stage_2_Hypertension',
                            np.where((combined.sysbp_invasive_d1_avg >= 130) | 
                                (combined.sysbp_noninvasive_d1_avg >= 130) |
                                (combined.diasbp_invasive_d1_avg >= 80) | 
                                (combined.diasbp_noninvasive_d1_avg >= 80), 'Stage_1_Hypertension',
                            np.where((combined.sysbp_invasive_d1_avg < 90) | 
                                (combined.sysbp_noninvasive_d1_avg < 90) |
                                (combined.diasbp_invasive_d1_avg < 60) | 
                                (combined.diasbp_noninvasive_d1_avg < 60), 'Hypotension',
                            np.where((combined.sysbp_invasive_d1_avg >= 120) | 
                                (combined.sysbp_noninvasive_d1_avg >= 120), 'Eleveated_BP','Normal_BP')))))
combined['a1c_bp_group'] = np.where(combined['bp_group'] == 'Not_Measured', 'NoBP',
                                    combined['a1c_group'] + '_' + combined['bp_group'])

### Create aggregated mean and index lab test features for each grouping feature 

In [ ]:
for grouper in ['apache_2_diagnosis','demo_profile','a1c_bp_group']:
    for lab in list(combined.columns[combined.columns.str.endswith('avg')])+['weight','bmi','age']:
        mean_name = '{0}_{1}_mean'.format(grouper,lab)
        combined[mean_name] = combined[grouper].map(combined.groupby(grouper)[lab].mean())
        combined[mean_name+'_diff'.format(grouper,lab)] = combined[lab] - combined[mean_name]
        combined[mean_name+'_ratio'.format(grouper,lab)] = np.maximum(np.minimum(
            combined[lab] / combined[mean_name], RATIO_CAP), 0)

### Frequency encoding of categorical features

In [ ]:
cat_cols = ['ethnicity','gender','hospital_admit_source','icu_admit_source','icu_stay_type','icu_type',
            'weight_category','age_group','demo_profile','apache_3j_group','a1c_group','bp_group','a1c_bp_group',
            'apache_2_diagnosis','apache_3j_diagnosis']
for cat in cat_cols:
    combined['{0}_freq'.format(cat)] = np.log1p(combined[cat].map(combined[cat].value_counts()))

In [ ]:
# cols_to_keep = list(pd.read_csv('Data/test_FE2_filter5.csv').columns)
# cols_to_keep += ['dataset_label',TARGET_COL]
# combined = reduce_mem_usage(combined[cols_to_keep])

# Clean up and split back into train and test set

In [ ]:
# Detect and remove duplicate features
dropped_cols = set(combined.columns) - set(combined.T.drop_duplicates().index)

In [ ]:
combined = reduce_mem_usage(combined.drop(dropped_cols,axis=1))

In [ ]:
train_out = combined[combined.dataset_label == 0].drop('dataset_label',axis=1)
test_out = combined[combined.dataset_label == 1].drop(['dataset_label', TARGET_COL],axis=1)

In [ ]:
print(test_out.shape)
print(test_raw.shape)
print(train_out.shape)
print(train_raw.shape)

In [ ]:
train_out.to_csv('Data/train_FE2_apache.csv',index=False)
test_out.to_csv('Data/test_FE2_apache.csv',index=False)
# train_out.to_csv('Data/train_FE2_filter5_ratio_cap4.csv',index=False)
# test_out.to_csv('Data/test_FE2_filter5_ratio_cap4.csv',index=False)